## Manage Database Credentials using GCP Secret Manager

Here is the process involved to get secret details as part of the applications.
* Create Secret Manager Client Object
* Get Secret Details
* Use Secret Details (to connect to Databases)

In [1]:
# from google.cloud import secretmanager
from google.cloud import secretmanager 

/home/sammigachuhi/itversity/data-engineering-on-gcp/deg-venv/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/home/sammigachuhi/itversity/data-engineering-on-gcp/deg-venv/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.cloud.secretmanager_v1 once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.cloud.secretmanager_v1 past that date.
  warnings.warn(message, FutureWarning)


In [2]:
# client = secretmanager.SecretManagerServiceClient()
client = secretmanager.SecretManagerServiceClient()

In [3]:
# client.access_secret_version?
client.access_secret_version?

Signature:
client.access_secret_version(
    request: Union[google.cloud.secretmanager_v1.types.service.AccessSecretVersionRequest, dict, NoneType] = None,
    *,
    name: Optional[str] = None,
    retry: Union[google.api_core.retry.retry_unary.Retry, google.api_core.gapic_v1.method._MethodDefault, NoneType] = <_MethodDefault._DEFAULT_VALUE: <object object at 0x7a40e4514240>>,
    timeout: Union[float, object] = <_MethodDefault._DEFAULT_VALUE: <object object at 0x7a40e4514240>>,
    metadata: Sequence[Tuple[str, Union[str, bytes]]] = (),
) -> google.cloud.secretmanager_v1.types.service.AccessSecretVersionResponse
Docstring:
Accesses a
[SecretVersion][google.cloud.secretmanager.v1.SecretVersion].
This call returns the secret data.

``projects/*/secrets/*/versions/latest`` is an alias to the most
recently created
[SecretVersion][google.cloud.secretmanager.v1.SecretVersion].

.. code-block:: python

    # This snippet has been automatically generated and should be regarded as a
    # cod

In [4]:
# project_id = 'tidy-fort-361710'
# secret_id = 'retailsecret'
# version_id = 1
project_id = 'data-engineering-gcp-483907'
secret_id = 'retailsecret'
version_id = 1 


In [5]:
# secret_name = f'projects/{project_id}/secrets/{secret_id}/versions/{version_id}'
secret_name = f'projects/{project_id}/secrets/{secret_id}/versions/{version_id}'

In [6]:
# secret_name
secret_name

'projects/data-engineering-gcp-483907/secrets/retailsecret/versions/1'

In [7]:
# response = client.access_secret_version(name=secret_name)
response = client.access_secret_version(name=secret_name)

In [9]:
response

name: "projects/277371599402/secrets/retailsecret/versions/1"
payload {
  data: "{\"host\": \"104.197.141.244\",\n\"database\": \"itversity_retail_db\", \n\"user\": \"itversity_retail_user\", \n\"password\": \"itversity\"}"
  data_crc32c: 3007144142
}

In [8]:
type(response)

google.cloud.secretmanager_v1.types.service.AccessSecretVersionResponse

In [10]:
# response.payload
response.payload

data: "{\"host\": \"104.197.141.244\",\n\"database\": \"itversity_retail_db\", \n\"user\": \"itversity_retail_user\", \n\"password\": \"itversity\"}"
data_crc32c: 3007144142

In [11]:
# response.payload.data
response.payload.data

b'{"host": "104.197.141.244",\n"database": "itversity_retail_db", \n"user": "itversity_retail_user", \n"password": "itversity"}'

In [12]:
# response.payload.data.decode('utf-8')
response.payload.data.decode('utf-8')

'{"host": "104.197.141.244",\n"database": "itversity_retail_db", \n"user": "itversity_retail_user", \n"password": "itversity"}'

In [13]:
# import json
import json

In [14]:
# secret_details = json.loads(response.payload.data.decode('utf-8'))
secret_details = json.loads(response.payload.data.decode('utf-8'))

In [15]:
# secret_details.keys()
secret_details.keys()

dict_keys(['host', 'database', 'user', 'password'])

In [16]:
# secret_details['user']
secret_details['user']

'itversity_retail_user'

In [18]:
# secret_details['password']
secret_details['password']

'itversity'

In [19]:
# import json
# from google.cloud import secretmanager

# def get_secret_details(secret_name):
#     client = secretmanager.SecretManagerServiceClient()
#     response = client.access_secret_version(name=secret_name)
#     return json.loads(response.payload.data.decode('utf-8'))

import json  
from google.cloud import secretmanager 

def get_secret_details(secret_name):
    client = secretmanager.SecretManagerServiceClient()
    response = client.access_secret_version(name=secret_name) 
    return json.loads(response.payload.data.decode('utf-8'))

In [20]:
# project_id = 'tidy-fort-361710'
# secret_id = 'retailsecret'
# version_id = 1

# secret_name = f'projects/{project_id}/secrets/{secret_id}/versions/{version_id}'
secret_details = get_secret_details(secret_name)

In [21]:
secret_details

{'host': '104.197.141.244',
 'database': 'itversity_retail_db',
 'user': 'itversity_retail_user',
 'password': 'itversity'}

In [ ]:
conn_uri = 'postgresql://{user}:{password}@{host}:{port}/{database}'

In [ ]:
conn_uri.format(port=5432, **secret_details)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_sql(
    '''
        SELECT order_status, sum(order_count) AS order_count FROM daily_status_count
        GROUP BY 1
        ORDER BY 2 DESC
    ''',
    conn_uri.format(port=5432, **secret_details)
)

In [ ]:
df